# SUMMARY

This notebook reproduces my submission that scores **0.971** on private LB and reaches the **71st** place. The notebook implements ensemble of 5+2 CNN models using pretrained weights saved as Kaggle datasets.

![ensemble](https://i.postimg.cc/c4cPcXng/ranzcr.png)
- A detailed summary of my solution is published [in this discussion topic](https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/discussion/226664)
- Complete codes and notebooks for base models will soon be available on [my GitHub](https://github.com/kozodoi)

# PARAMETERS

In [1]:
####### NOTEBOOK CONFIGURATION

class CFG:
    
    # environment
    device      = 'GPU'  # device ['CPU', 'GPU']
    cpu_workers = 2      # no. cores
    debug       = True   # debug runs inference on a single batch
    seed        = 13353  # random state

    # inference
    batch_size  = 50     # no. images per batch
    num_tta     = 2      # no. TTA flips (between 1 and 8)
    num_folds   = 3      # no. folds per model (between 1 and 5)
    fold_idx    = False  # load weights from one fold [False, fold index]
      
    # blending
    fold_blend  = 'pmean'  # how to blend folds  ['amean', 'median', 'gmean', 'pmean', 'rmean']
    model_blend = 'pmean'  # how to blend models ['amean', 'median', 'gmean', 'pmean', 'rmean']
    power       = 1/11     # power parameter for pmean
    w_public    = 0.25     # weight of public models in the final blend   

    # stacking
    lgb_folds       = 5      # no. folds for stacking
    label_features  = False  # use only label-specific features
    sort_targets    = True   # sort targets by AUC
    pred_as_feature = True   # include class predictions as features for other classes
    lgb_stop_rounds = 200    # no. early stopping rounds
    lgb_params      = {'objective':         'binary',
                       'metrics':           'auc',
                       'n_estimators':      10000,
                       'learning_rate':     0.01,
                       'num_leaves':        8,
                       'max_depth':         5,
                       'min_child_samples': 20,
                       'subsample':         0.3,
                       'colsample_bytree':  0.5,
                       'reg_alpha':         0.1,
                       'reg_lambda':        0.1,
                       'silent':            True,
                       'verbosity':         -1,
                       'n_jobs' :           -1,
                       'random_state':      13353}

    # paths
    data_path = '/kaggle/input/ranzcr-clip-catheter-line-classification/'
    
    # list of models
    models = ['/kaggle/input/ranzcr-v12/', 
              '/kaggle/input/ranzcr-v15-pub/', 
              '/kaggle/input/ranzcr-v17-pub/', 
              '/kaggle/input/ranzcr-v13-pub/', 
              '/kaggle/input/ranzcr-v14-pub/']

In [2]:
####### CONVERT CONFIGURATION

CFG = dict(vars(CFG))
for key in ['__dict__', '__doc__', '__module__', '__weakref__']:
    del CFG[key]

During training, I save a `pickle` configuration file for each base model to make it convenient to import their weights, parameters and image sizes during inference. Below I load all model configurations in a list.

In [3]:
####### LOAD MODEL CONFIGURATIONS

import pickle

CFGs = []

for model in CFG['models']:
    model_cfg = pickle.load(open(model + 'configuration.pkl', 'rb'))
    CFGs.append(model_cfg)
    
print('Numer of models:', len(CFGs))

Numer of models: 5


# PREPARATIONS

In [4]:
####### PACKAGES

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler

import albumentations as A
from albumentations.pytorch import ToTensorV2

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2

from scipy.stats import rankdata, gmean
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold

import lightgbm as lgb

from tqdm import tqdm

import random
import time
import sys
import os

import gc

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [5]:
####### TRAINING DEVICE
    
if CFG['device'] == 'GPU':
    print('Training on GPU...')
    device = torch.device('cuda:0')

if CFG['device'] == 'CPU':
    print('Training on CPU...')
    device = torch.device('cpu') 

Training on GPU...


The function `compute_blend()` implements simple blends (arithmetic, geometric, power and rank mean).

In [6]:
####### UTILITIES

# competition metric
def get_score(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[1]):
        score = roc_auc_score(y_true[:,i], y_pred[:,i])
        scores.append(score)
    avg_score = np.mean(scores)
    return avg_score, scores

# simple ensembles
def compute_blend(df, preds, blend, CFG, weights = None):
    
    if weights is None:
        weights = np.ones(len(preds)) / len(preds)
        
    if blend == 'amean':
        out = np.sum(df[preds].values * weights, axis = 1)
    elif blend == 'median':
        out = df[preds].median(axis = 1)
    elif blend == 'gmean':
        out = np.prod(np.power(df[preds].values, weights), axis = 1)
    elif blend == 'pmean':
        out = np.sum(np.power(df[preds].values, CFG['power']) * weights, axis = 1) ** (1 / CFG['power'])
    elif blend == 'rmean':
        out = np.sum(df[preds].rank(pct = True).values * weights, axis = 1)
    return out

# randomness
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    print('setting random seed to {}...'.format(seed))
    
seed_everything(CFG['seed'])

setting random seed to 13353...


# DATA PREP

To save GPU quota, I was subsetting the test images to a single batch when commiting the notebook. During submission, the condition `len(df) == 3582` is not fulfilled and the code is run on the full test set.

In [7]:
####### DATA PREPARATION

# import data
df = pd.read_csv(CFG['data_path'] + 'sample_submission.csv')

# num classes
CFG['targets'] = ['ETT - Abnormal', 
                  'ETT - Borderline', 
                  'ETT - Normal', 
                  'NGT - Abnormal', 
                  'NGT - Borderline', 
                  'NGT - Incompletely Imaged', 
                  'NGT - Normal', 
                  'CVC - Abnormal',
                  'CVC - Borderline', 
                  'CVC - Normal', 
                  'Swan Ganz Catheter Present']
CFG['num_classes'] = len(CFG['targets'])

# debug mode
if CFG['debug'] and len(df) == 3582:
    print('Subsetting data in the debug mode...')
    df = df.head(CFG['batch_size'])
    
print(df.shape)

Subsetting data in the debug mode...
(50, 12)


To evaluate OOF performance, I was averaging AUCs on validation folds instead of computing OOF AUC on the full training data. Since AUC is a ranking indicator, this way to evaluate performance provided a more robust CV/LB correlation.

In [8]:
####### CHECK OOF PERFORMANCE

# load preds
for m in CFG['models']:

    tmp_train_preds         = pd.read_csv(m + '/oof.csv')
    tmp_train_preds.columns = ['StudyInstanceUID'] + CFG['targets'] + ['PatientID', 'fold'] + [m + ' ' + c for c in CFG['targets']]

    if m == CFG['models'][0]:
        train_preds = tmp_train_preds        
    else:
        train_preds = train_preds.merge(tmp_train_preds[['StudyInstanceUID'] + [m + ' ' + c for c in CFG['targets']]], how = 'left', on = 'StudyInstanceUID')

# sort models by performance
weights = []
for model_idx, m in enumerate(CFG['models']):
    score = 0
    for fold_idx in range(5):
        tmp_train_preds = train_preds.loc[train_preds['fold'] == fold_idx]
        score += get_score(tmp_train_preds[CFG['targets']].values, tmp_train_preds[[m + ' ' + c for c in CFG['targets']]].values)[0] / 5
    weights.append(score)
sorted_ids     = list(np.argsort(np.array(weights)))
sorted_weights = [weights[i] for i in sorted_ids]
CFG['models']  = [CFG['models'][i] for i in sorted_ids]
CFGs           = [CFGs[i] for i in sorted_ids]

# display performance 
print('-' * 45)
print('{:<5}{:<33}{:>5}'.format('ID', 'Model', 'AUC'))
print('-' * 45)
for model_idx, m in enumerate(CFG['models']):
    print('{:<5}{:<33}{:.4f}'.format(model_idx + 1, m, sorted_weights[model_idx]))
print('-' * 45)

---------------------------------------------
ID   Model                              AUC
---------------------------------------------
1    /kaggle/input/ranzcr-v12/        0.9521
2    /kaggle/input/ranzcr-v15-pub/    0.9601
3    /kaggle/input/ranzcr-v17-pub/    0.9608
4    /kaggle/input/ranzcr-v13-pub/    0.9611
5    /kaggle/input/ranzcr-v14-pub/    0.9618
---------------------------------------------


Below I test performance of different simple ensembles on OOF predictions. Unweighted power mean with `power = 1/11` worked best in my experiments.

In [9]:
####### CHECK BLEND PERFORMANCE

# compute predcitions
for c in CFG['targets']:   
    
    class_preds = train_preds.filter(like = 'kaggle').filter(like = c).columns
    
    for blend in ['amean', 'median', 'gmean', 'pmean', 'rmean']:
        train_preds[blend + ' ' + c] = compute_blend(train_preds, class_preds, blend, CFG)
        
    for blend in ['amean', 'median', 'gmean', 'pmean', 'rmean']:
        train_preds['w' + blend + ' ' + c] = compute_blend(train_preds, class_preds, blend, CFG, weights = np.array(sorted_weights))
    
# compute performance
print('-' * 18)
print('{:<10}{:>5}'.format('Blend', 'AUC'))
print('-' * 18)
for blend in ['amean', 'median', 'gmean', 'pmean', 'rmean']:
    score = 0
    for fold_idx in range(5):
        tmp_train_preds = train_preds.loc[train_preds['fold'] == fold_idx]
        score += get_score(tmp_train_preds[CFG['targets']].values, tmp_train_preds[[blend + ' ' + c for c in CFG['targets']]].values)[0] / 5
    print('{:<10}{:>5.4f}'.format(blend, score))    
print('-' * 18)
for blend in ['amean', 'median', 'gmean', 'pmean', 'rmean']:
    score = 0
    for fold_idx in range(5):
        tmp_train_preds = train_preds.loc[train_preds['fold'] == fold_idx]
        score += get_score(tmp_train_preds[CFG['targets']].values, tmp_train_preds[['w' + blend + ' ' + c for c in CFG['targets']]].values)[0] / 5
    print('{:<10}{:>5.4f}'.format('w' + blend, score))    
print('-' * 18)

------------------
Blend       AUC
------------------
amean     0.9657
median    0.9649
gmean     0.9658
pmean     0.9658
rmean     0.9655
------------------
wamean    0.9657
wmedian   0.9649
wgmean    0.9658
wpmean    0.9658
wrmean    0.9655
------------------


# IMAGE PROCESSING

In [10]:
####### DATASET

def get_dataset(CFG):

    class ImageData(Dataset):

        # initialization
        def __init__(self, 
                     df, 
                     path, 
                     transform = None, 
                     labeled   = False,
                     indexed   = False):
            self.df        = df
            self.path      = path
            self.transform = transform
            self.labeled   = labeled
            self.indexed   = indexed

        # length
        def __len__(self):
            return len(self.df)

        # get item  
        def __getitem__(self, idx):

            # import
            path  = os.path.join(self.path, self.df.iloc[idx]['StudyInstanceUID'] + '.jpg')
            image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                raise FileNotFoundError(path)
        
            # crop black borders
            '''
            Borrowed from https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/discussion/224146
            '''
            mask  = image > 0
            image = image[np.ix_(mask.any(1), mask.any(0))]

            # convert to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # augmentations
            if self.transform is not None:
                image = self.transform(image = image)['image']
                
            # transform 
            if CFG['normalize'] == 'public':
                image = image.astype(np.float32)
                image = image.transpose(2, 0, 1)
                image = torch.tensor(image).float()

            # output
            if self.labeled:
                label = torch.tensor(self.df.iloc[idx][CFG['targets']]).float()
                if self.indexed:
                    idx = torch.tensor(idx)
                    return idx, image, label
                else: 
                    return image, label
            else:
                return image
            
    return ImageData

In [11]:
####### AUGMENTATIONS

def get_augs(CFG, image_size = None):

    if CFG['normalize'] != 'public':
        test_augs = A.Compose([A.Resize(height = image_size, 
                                        width  = image_size),
                               A.Normalize(mean = CFG['pixel_mean'],
                                           std  = CFG['pixels_std']),
                               ToTensorV2()])
    else:
        test_augs = A.Compose([A.Resize(height = image_size, 
                                        width  = image_size),
                               A.Normalize()])

    # output
    return test_augs

In [12]:
####### TTA HELPER FUNCTION

'''
Based on https://github.com/haqishen/SIIM-ISIC-Melanoma-Classification-1st-Place-Solution
'''
def get_tta_flips(img, i):

    if i >= 4:
        img = img.transpose(2, 3)
    if i % 4 == 0:
        return img
    elif i % 4 == 1:
        return img.flip(3)
    elif i % 4 == 2:
        return img.flip(2)
    elif i % 4 == 3:
        return img.flip(3).flip(2)

# MODEL PREP

All models were imported from the `timm` library. I used [this dataset](https://www.kaggle.com/kozodoi/timm-pytorch-image-models) with the most recent version of `timm`.

In [13]:
####### MODEL ARCHITECTURE

def get_model(CFG, device, num_classes):
    
    if CFG['weights'] != 'public':

        model = timm.create_model(model_name = CFG['backbone'], 
                                  pretrained = False,
                                  in_chans   = CFG['channels'])

        if 'efficient' in CFG['backbone']:
            model.classifier = nn.Linear(model.classifier.in_features, num_classes)
        else:
            model.fc = nn.Linear(model.fc.in_features, num_classes)

    else:
        
        class CustomModel(nn.Module):
            
            def __init__(self, model_name = 'resnet200d', out_dim = 11, pretrained = False):
                super().__init__()
                self.model             = timm.create_model(model_name, pretrained=False)
                n_features             = self.model.fc.in_features
                self.model.global_pool = nn.Identity()
                self.model.fc          = nn.Identity()
                self.pooling           = nn.AdaptiveAvgPool2d(1)
                self.fc                = nn.Linear(n_features, out_dim)

            def forward(self, x):
                bs              = x.size(0)
                features        = self.model(x)
                pooled_features = self.pooling(features).view(bs, -1)
                output          = self.fc(pooled_features)
                return output

        model = CustomModel(CFG['backbone'], num_classes, True)
    
        
    return model

# INFERENCE

The inference loop goes through 5 CNN models and stacks their predictions in a pandas dataframe. For each model, I also loop through the specified number of folds and iterate fold indices between 1 and 5.

In [14]:
########## INFERENCE FOR NORMAL MODELS

# timer
cv_start = time.time()
gc.collect()

# counter
all_counter  = 0
fold_counter = 0 if not CFG['fold_idx'] else CFG['fold_idx']

# placeholder
all_cnn_preds = None

# loop through models
for model_idx in range(len(CFG['models'])):
           
    # data prep
    ImageData    = get_dataset(CFGs[model_idx])
    test_dataset = ImageData(df        = df, 
                             path      = CFG['data_path'] + 'test/',
                             transform = get_augs(CFGs[model_idx], image_size = CFGs[model_idx]['image_size']),
                             labeled   = False,
                             indexed   = False)
    test_loader = DataLoader(test_dataset, 
                             batch_size  = CFG['batch_size'], 
                             shuffle     = False, 
                             num_workers = CFG['cpu_workers'],
                             pin_memory  = True)
    
    # loop thgough folds
    for fold_idx in tqdm(range(CFG['num_folds'])):
           
        # model prep
        model = get_model(CFGs[model_idx], device = device, num_classes = CFG['num_classes'])
        model = model.to(device)
        model.load_state_dict(torch.load(CFG['models'][model_idx] + 'weights_fold{}.pth'.format(fold_counter),map_location = device))
        model.eval()

        # probs placeholder
        PROBS = []

        # loop through batches 
        with torch.no_grad():
            for batch_idx, inputs in enumerate(test_loader):

                # extract inputs 
                inputs = inputs.to(device)
                
                # preds placeholders
                probs = torch.zeros((inputs.shape[0], CFG['num_classes']), device = device)

                # inference with TTA
                for tta_idx in range(CFG['num_tta']): 
                    preds  = model(get_tta_flips(inputs, tta_idx))
                    probs += preds.sigmoid()

                # store predictions
                PROBS.append(probs.detach().cpu() / CFG['num_tta'])
                
        # store predictions
        cnn_preds     = pd.DataFrame(torch.cat(PROBS).numpy(), columns = [CFG['models'][model_idx] + str(fold_idx) + '/' + c for c in CFG['targets']])
        all_cnn_preds = pd.concat([all_cnn_preds, cnn_preds], axis = 1)

        # update counters
        all_counter += 1
        if not CFG['fold_idx']:
            fold_counter += 1
            if fold_counter == CFG['num_folds']:
                fold_counter = 0

        # clear memory
        del model, inputs, preds, probs, PROBS
        gc.collect()
        
    # clear memory
    del test_loader, test_dataset
    gc.collect()
        
# print performance
print('Finished {} preds x {} TTA in {:.2f} hours'.format(all_counter, CFG['num_tta'], (time.time() - cv_start) / 3600))

100%|██████████| 3/3 [00:50<00:00, 16.78s/it]

Finished 15 preds x 2 TTA in 0.07 hours


In [15]:
####### BLEND FOLD PREDICTIONS

print('Blending fold predictions with: ' + CFG['fold_blend'])

for m in CFG['models']:
    for c in CFG['targets']:
        class_preds = all_cnn_preds.filter(like = m).filter(like = c).columns
        all_cnn_preds[m + c] = compute_blend(all_cnn_preds, class_preds, CFG['fold_blend'], CFG)
        all_cnn_preds.drop(class_preds, axis = 1, inplace = True)
           
all_cnn_preds.head()

Blending fold predictions with: pmean


,/kaggle/input/ranzcr-v12/ETT - Abnormal,/kaggle/input/ranzcr-v12/ETT - Borderline,/kaggle/input/ranzcr-v12/ETT - Normal,/kaggle/input/ranzcr-v12/NGT - Abnormal,/kaggle/input/ranzcr-v12/NGT - Borderline,/kaggle/input/ranzcr-v12/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v12/NGT - Normal,/kaggle/input/ranzcr-v12/CVC - Abnormal,/kaggle/input/ranzcr-v12/CVC - Borderline,/kaggle/input/ranzcr-v12/CVC - Normal,/kaggle/input/ranzcr-v12/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v15-pub/ETT - Abnormal,/kaggle/input/ranzcr-v15-pub/ETT - Borderline,/kaggle/input/ranzcr-v15-pub/ETT - Normal,/kaggle/input/ranzcr-v15-pub/NGT - Abnormal,/kaggle/input/ranzcr-v15-pub/NGT - Borderline,/kaggle/input/ranzcr-v15-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v15-pub/NGT - Normal,/kaggle/input/ranzcr-v15-pub/CVC - Abnormal,/kaggle/input/ranzcr-v15-pub/CVC - Borderline,/kaggle/input/ranzcr-v15-pub/CVC - Normal,/kaggle/input/ranzcr-v15-pub/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v17-pub/ETT - Abnormal,/kaggle/input/ranzcr-v17-pub/ETT - Borderline,/kaggle/input/ranzcr-v17-pub/ETT - Normal,/kaggle/input/ranzcr-v17-pub/NGT - Abnormal,/kaggle/input/ranzcr-v17-pub/NGT - Borderline,/kaggle/input/ranzcr-v17-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v17-pub/NGT - Normal,/kaggle/input/ranzcr-v17-pub/CVC - Abnormal,/kaggle/input/ranzcr-v17-pub/CVC - Borderline,/kaggle/input/ranzcr-v17-pub/CVC - Normal,/kaggle/input/ranzcr-v17-pub/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v13-pub/ETT - Abnormal,/kaggle/input/ranzcr-v13-pub/ETT - Borderline,/kaggle/input/ranzcr-v13-pub/ETT - Normal,/kaggle/input/ranzcr-v13-pub/NGT - Abnormal,/kaggle/input/ranzcr-v13-pub/NGT - Borderline,/kaggle/input/ranzcr-v13-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v13-pub/NGT - Normal,/kaggle/input/ranzcr-v13-pub/CVC - Abnormal,/kaggle/input/ranzcr-v13-pub/CVC - Borderline,/kaggle/input/ranzcr-v13-pub/CVC - Normal,/kaggle/input/ranzcr-v13-pub/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v14-pub/ETT - Abnormal,/kaggle/input/ranzcr-v14-pub/ETT - Borderline,/kaggle/input/ranzcr-v14-pub/ETT - Normal,/kaggle/input/ranzcr-v14-pub/NGT - Abnormal,/kaggle/input/ranzcr-v14-pub/NGT - Borderline,/kaggle/input/ranzcr-v14-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v14-pub/NGT - Normal,/kaggle/input/ranzcr-v14-pub/CVC - Abnormal,/kaggle/input/ranzcr-v14-pub/CVC - Borderline,/kaggle/input/ranzcr-v14-pub/CVC - Normal,/kaggle/input/ranzcr-v14-pub/Swan Ganz Catheter Present
0,0.020610,0.329487,0.520409,0.000740,0.001070,5.301902e-03,0.992334,0.026280,0.262628,0.787519,0.999553,0.005210,0.140301,0.694103,0.003112,0.004801,0.039867,0.969904,0.031317,0.078975,0.885068,0.997096,0.003072,0.154419,0.775875,0.001489,0.003276,0.040277,0.970417,0.035951,0.096740,0.876043,0.999412,0.020450,0.436476,0.477290,0.001034,0.002475,0.019706,0.976903,0.029734,0.080187,0.856435,0.998158,0.009721,0.284677,0.640569,0.001115,0.002327,0.019956,0.978515,0.023542,0.071492,0.917398,0.999833
1,0.000002,0.000002,0.000004,0.000014,0.000023,6.258607e-07,0.000007,0.017637,0.002424,0.994523,0.000002,0.000009,0.000025,0.000228,0.000048,0.000089,0.000049,0.000124,0.005564,0.005606,0.993592,0.000002,0.000025,0.000163,0.000869,0.000106,0.000208,0.000138,0.000306,0.001836,0.005760,0.995082,0.000022,0.000008,0.000037,0.000291,0.000046,0.000071,0.000032,0.000084,0.001955,0.007289,0.988535,0.000004,0.000018,0.000112,0.000760,0.000084,0.000132,0.000094,0.000216,0.001497,0.004033,0.996565,0.000009
2,0.000005,0.000075,0.000088,0.000074,0.000097,3.603585e-05,0.000045,0.003033,0.442185,0.534228,0.000004,0.000044,0.000064,0.000166,0.000144,0.000217,0.000131,0.000203,0.006740,0.424880,0.477583,0.000033,0.000109,0.000235,0.000424,0.000255,0.000258,0.000278,0.000326,0.009818,0.504654,0.490681,0.000254,0.000055,0.000097,0.000147,0.000158,0.000186,0.000169,0.000158,0.010998,0.539751,0.412494,0.000117,0.000090,0.000190,0.000386,0.000272,0.000348,0.000223,0.000293,0.008109,0.601265,0.358833,0.000130
3,0.0

# STACKING

First, I load OOF predictions from CNNs and make sure train / test dataframes have the same format.

In [16]:
####### PREPARE OOF PREDS

for m in CFG['models']:

    tmp_train_preds         = pd.read_csv(m + '/oof.csv')
    tmp_train_preds.columns = ['StudyInstanceUID'] + CFG['targets'] + ['PatientID', 'fold'] + [m + '' + c for c in CFG['targets']]

    if m == CFG['models'][0]:
        train_preds = tmp_train_preds        
    else:
        train_preds = train_preds.merge(tmp_train_preds[['StudyInstanceUID'] + [m + '' + c for c in CFG['targets']]], how = 'left', on = 'StudyInstanceUID')

train_preds.head()

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID,fold,/kaggle/input/ranzcr-v12/ETT - Abnormal,/kaggle/input/ranzcr-v12/ETT - Borderline,/kaggle/input/ranzcr-v12/ETT - Normal,/kaggle/input/ranzcr-v12/NGT - Abnormal,/kaggle/input/ranzcr-v12/NGT - Borderline,/kaggle/input/ranzcr-v12/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v12/NGT - Normal,/kaggle/input/ranzcr-v12/CVC - Abnormal,/kaggle/input/ranzcr-v12/CVC - Borderline,/kaggle/input/ranzcr-v12/CVC - Normal,/kaggle/input/ranzcr-v12/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v15-pub/ETT - Abnormal,/kaggle/input/ranzcr-v15-pub/ETT - Borderline,/kaggle/input/ranzcr-v15-pub/ETT - Normal,/kaggle/input/ranzcr-v15-pub/NGT - Abnormal,/kaggle/input/ranzcr-v15-pub/NGT - Borderline,/kaggle/input/ranzcr-v15-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v15-pub/NGT - Normal,/kaggle/input/ranzcr-v15-pub/CVC - Abnormal,/kaggle/input/ranzcr-v15-pub/CVC - Borderline,/kaggle/input/ranzcr-v15-pub/CVC - Normal,/kaggle/input/ranzcr-v15-pub/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v17-pub/ETT - Abnormal,/kaggle/input/ranzcr-v17-pub/ETT - Borderline,/kaggle/input/ranzcr-v17-pub/ETT - Normal,/kaggle/input/ranzcr-v17-pub/NGT - Abnormal,/kaggle/input/ranzcr-v17-pub/NGT - Borderline,/kaggle/input/ranzcr-v17-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v17-pub/NGT - Normal,/kaggle/input/ranzcr-v17-pub/CVC - Abnormal,/kaggle/input/ranzcr-v17-pub/CVC - Borderline,/kaggle/input/ranzcr-v17-pub/CVC - Normal,/kaggle/input/ranzcr-v17-pub/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v13-pub/ETT - Abnormal,/kaggle/input/ranzcr-v13-pub/ETT - Borderline,/kaggle/input/ranzcr-v13-pub/ETT - Normal,/kaggle/input/ranzcr-v13-pub/NGT - Abnormal,/kaggle/input/ranzcr-v13-pub/NGT - Borderline,/kaggle/input/ranzcr-v13-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v13-pub/NGT - Normal,/kaggle/input/ranzcr-v13-pub/CVC - Abnormal,/kaggle/input/ranzcr-v13-pub/CVC - Borderline,/kaggle/input/ranzcr-v13-pub/CVC - Normal,/kaggle/input/ranzcr-v13-pub/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v14-pub/ETT - Abnormal,/kaggle/input/ranzcr-v14-pub/ETT - Borderline,/kaggle/input/ranzcr-v14-pub/ETT - Normal,/kaggle/input/ranzcr-v14-pub/NGT - Abnormal,/kaggle/input/ranzcr-v14-pub/NGT - Borderline,/kaggle/input/ranzcr-v14-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v14-pub/NGT - Normal,/kaggle/input/ranzcr-v14-pub/CVC - Abnormal,/kaggle/input/ranzcr-v14-pub/CVC - Borderline,/kaggle/input/ranzcr-v14-pub/CVC - Normal,/kaggle/input/ranzcr-v14-pub/Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814,0,0.000081,0.000047,0.000087,0.000892,0.000780,1.428677e-04,0.000162,0.590756,0.421370,0.086591,0.000082,0.000163,0.000209,0.000597,0.002947,0.001815,0.001025,0.000570,0.984873,0.014158,0.019166,1.751979e-04,0.000064,0.000174,0.000599,0.001180,0.001550,0.000446,0.000418,0.974239,0.037076,0.019774,0.000502,0.000184,0.000288,0.001259,0.003099,0.004118,0.001514,0.001379,0.993248,0.014276,0.014515,0.000245,0.000129,0.000244,0.000547,0.002013,0.002477,0.001256,0.000802,0.988044,0.034508,0.014954,0.000493
1,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1,0,0.000039,0.000105,0.000076,0.000658,0.000101,5.091655e-04,0.000042,0.034454,0.004401,0.978490,0.000025,0.000005,0.000070,0.000079,0.000018,0.000021,0.000039,0.000038,0.000116,0.001292,0.999547,5.478188e-07,0.000020,0.000161,0.000229,0.000075,0.000080,0.000052,0.000057,0.001528,0.008322,0.997434,0.000029,0.000017,0.000145,0.000288,0.000068,0.000078,0.000090,0.000103,0.000474,0.002843,0.997282,0.000009,0.000004,0.000162,0.002281,0.000047,0.000057,0.000103,0.000059,0.000148,0.000959,0.999219,0.000003
2,1.2.826.0.1.3680043.8.498.71008408169501434503...,0,0,0,0,0,0,0,0,0,1,0,7d2c4c9e3,0,0.000016,0.000007,0.0000

In [17]:
####### PREPARE TEST PREDS

test_preds = all_cnn_preds.copy()
test_preds = pd.concat([df['StudyInstanceUID'], test_preds], axis = 1)    
test_preds.head()

,StudyInstanceUID,/kaggle/input/ranzcr-v12/ETT - Abnormal,/kaggle/input/ranzcr-v12/ETT - Borderline,/kaggle/input/ranzcr-v12/ETT - Normal,/kaggle/input/ranzcr-v12/NGT - Abnormal,/kaggle/input/ranzcr-v12/NGT - Borderline,/kaggle/input/ranzcr-v12/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v12/NGT - Normal,/kaggle/input/ranzcr-v12/CVC - Abnormal,/kaggle/input/ranzcr-v12/CVC - Borderline,/kaggle/input/ranzcr-v12/CVC - Normal,/kaggle/input/ranzcr-v12/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v15-pub/ETT - Abnormal,/kaggle/input/ranzcr-v15-pub/ETT - Borderline,/kaggle/input/ranzcr-v15-pub/ETT - Normal,/kaggle/input/ranzcr-v15-pub/NGT - Abnormal,/kaggle/input/ranzcr-v15-pub/NGT - Borderline,/kaggle/input/ranzcr-v15-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v15-pub/NGT - Normal,/kaggle/input/ranzcr-v15-pub/CVC - Abnormal,/kaggle/input/ranzcr-v15-pub/CVC - Borderline,/kaggle/input/ranzcr-v15-pub/CVC - Normal,/kaggle/input/ranzcr-v15-pub/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v17-pub/ETT - Abnormal,/kaggle/input/ranzcr-v17-pub/ETT - Borderline,/kaggle/input/ranzcr-v17-pub/ETT - Normal,/kaggle/input/ranzcr-v17-pub/NGT - Abnormal,/kaggle/input/ranzcr-v17-pub/NGT - Borderline,/kaggle/input/ranzcr-v17-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v17-pub/NGT - Normal,/kaggle/input/ranzcr-v17-pub/CVC - Abnormal,/kaggle/input/ranzcr-v17-pub/CVC - Borderline,/kaggle/input/ranzcr-v17-pub/CVC - Normal,/kaggle/input/ranzcr-v17-pub/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v13-pub/ETT - Abnormal,/kaggle/input/ranzcr-v13-pub/ETT - Borderline,/kaggle/input/ranzcr-v13-pub/ETT - Normal,/kaggle/input/ranzcr-v13-pub/NGT - Abnormal,/kaggle/input/ranzcr-v13-pub/NGT - Borderline,/kaggle/input/ranzcr-v13-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v13-pub/NGT - Normal,/kaggle/input/ranzcr-v13-pub/CVC - Abnormal,/kaggle/input/ranzcr-v13-pub/CVC - Borderline,/kaggle/input/ranzcr-v13-pub/CVC - Normal,/kaggle/input/ranzcr-v13-pub/Swan Ganz Catheter Present,/kaggle/input/ranzcr-v14-pub/ETT - Abnormal,/kaggle/input/ranzcr-v14-pub/ETT - Borderline,/kaggle/input/ranzcr-v14-pub/ETT - Normal,/kaggle/input/ranzcr-v14-pub/NGT - Abnormal,/kaggle/input/ranzcr-v14-pub/NGT - Borderline,/kaggle/input/ranzcr-v14-pub/NGT - Incompletely Imaged,/kaggle/input/ranzcr-v14-pub/NGT - Normal,/kaggle/input/ranzcr-v14-pub/CVC - Abnormal,/kaggle/input/ranzcr-v14-pub/CVC - Borderline,/kaggle/input/ranzcr-v14-pub/CVC - Normal,/kaggle/input/ranzcr-v14-pub/Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.020610,0.329487,0.520409,0.000740,0.001070,5.301902e-03,0.992334,0.026280,0.262628,0.787519,0.999553,0.005210,0.140301,0.694103,0.003112,0.004801,0.039867,0.969904,0.031317,0.078975,0.885068,0.997096,0.003072,0.154419,0.775875,0.001489,0.003276,0.040277,0.970417,0.035951,0.096740,0.876043,0.999412,0.020450,0.436476,0.477290,0.001034,0.002475,0.019706,0.976903,0.029734,0.080187,0.856435,0.998158,0.009721,0.284677,0.640569,0.001115,0.002327,0.019956,0.978515,0.023542,0.071492,0.917398,0.999833
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0.000002,0.000002,0.000004,0.000014,0.000023,6.258607e-07,0.000007,0.017637,0.002424,0.994523,0.000002,0.000009,0.000025,0.000228,0.000048,0.000089,0.000049,0.000124,0.005564,0.005606,0.993592,0.000002,0.000025,0.000163,0.000869,0.000106,0.000208,0.000138,0.000306,0.001836,0.005760,0.995082,0.000022,0.000008,0.000037,0.000291,0.000046,0.000071,0.000032,0.000084,0.001955,0.007289,0.988535,0.000004,0.000018,0.000112,0.000760,0.000084,0.000132,0.000094,0.000216,0.001497,0.004033,0.996565,0.000009
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0.000005,0.000075,0.000088,0.000074,0.000097,3.603585e-05,0.000045,0.003033,0.442185,0.534228,0.000004,0.000044,0.000064,0.000166,0.000144,0.000217,0.000131,0.000203,0.006740,0.424880,0.477583,0.000033,0.000109,0.000235,0.000424,0.000255,0.000258,0.000278,0.000326,0.009818,0.504654,0.490681,0.000254,0.000055,0.000097,0.000147,0.000158,

In [18]:
####### TRANSFORM DATA

X      = train_preds.copy()
X_test = test_preds.copy()

drop_features = ['StudyInstanceUID', 'PatientID', 'fold'] + CFG['targets']
features      = [f for f in X.columns if f not in drop_features]
print(len(features), 'features')
display(features)

55 features


['/kaggle/input/ranzcr-v12/ETT - Abnormal',
 '/kaggle/input/ranzcr-v12/ETT - Borderline',
 '/kaggle/input/ranzcr-v12/ETT - Normal',
 '/kaggle/input/ranzcr-v12/NGT - Abnormal',
 '/kaggle/input/ranzcr-v12/NGT - Borderline',
 '/kaggle/input/ranzcr-v12/NGT - Incompletely Imaged',
 '/kaggle/input/ranzcr-v12/NGT - Normal',
 '/kaggle/input/ranzcr-v12/CVC - Abnormal',
 '/kaggle/input/ranzcr-v12/CVC - Borderline',
 '/kaggle/input/ranzcr-v12/CVC - Normal',
 '/kaggle/input/ranzcr-v12/Swan Ganz Catheter Present',
 '/kaggle/input/ranzcr-v15-pub/ETT - Abnormal',
 '/kaggle/input/ranzcr-v15-pub/ETT - Borderline',
 '/kaggle/input/ranzcr-v15-pub/ETT - Normal',
 '/kaggle/input/ranzcr-v15-pub/NGT - Abnormal',
 '/kaggle/input/ranzcr-v15-pub/NGT - Borderline',
 '/kaggle/input/ranzcr-v15-pub/NGT - Incompletely Imaged',
 '/kaggle/input/ranzcr-v15-pub/NGT - Normal',
 '/kaggle/input/ranzcr-v15-pub/CVC - Abnormal',
 '/kaggle/input/ranzcr-v15-pub/CVC - Borderline',
 '/kaggle/input/ranzcr-v15-pub/CVC - Normal',
 '

Stacking is performed on OOFs using [group stratified 5-fold partitiioning](https://www.kaggle.com/underwearfitting/how-to-properly-split-folds) shared by [@underwearfitting](https://www.kaggle.com/underwearfitting). I was using the same partitinoing to train base models as well.

In [19]:
########## PARTITIONING

folds = pd.read_csv('/kaggle/input/how-to-properly-split-folds/train_folds.csv')
del X['fold']
X = X.merge(folds[['StudyInstanceUID', 'fold']], how = 'left', on = 'StudyInstanceUID')

Sorting targets by AUC helps to improve stacking performance. When starting with easier labels, models for more difficult labels have more features (that include OOF stacking predictions of previous labels).

In [20]:
####### SORT TARGETS BY AUC

if CFG['sort_targets']:

    sorted_targets = ['Swan Ganz Catheter Present',
                      'ETT - Normal', 
                      'ETT - Abnormal', 
                      'ETT - Borderline', 
                      'NGT - Abnormal', 
                      'NGT - Normal', 
                      'NGT - Incompletely Imaged', 
                      'NGT - Borderline', 
                      'CVC - Abnormal',
                      'CVC - Normal',
                      'CVC - Borderline']

For each label, I do the following:
- training a separate stacking model on 5 folds
- comparing performance of stacking with the best simple blend (power mean)
- saving predictions of the method that has a higher OOF AUC
- saving label predictions as a feature for the next label stacking model

In [21]:
########## STACKING

# placeholders
cnn_oof = np.zeros((len(X),      CFG['num_classes']))
lgb_oof = np.zeros((len(X),      CFG['num_classes']))
lgb_tst = np.zeros((len(X_test), CFG['lgb_folds'], CFG['num_classes']))
all_lgb_preds = None

# modeling loop
cv_start = time.time()
print('-' * 45)
print('{:<28}{:<7}{:>5}'.format('Label', 'Model', 'AUC'))
print('-' * 45)
for label in sorted_targets:

    # extract label
    y = X[label]

    # extract features
    label_features = [f for f in features if label in f] if CFG['label_features'] else features
    
    # placeholders
    lgb_auc = 0
    cnn_auc = 0

    # cross-validation
    for fold in range(CFG['lgb_folds']):
        
        # extract index
        trn_idx = X.loc[X['fold'] != fold].index
        val_idx = X.loc[X['fold'] == fold].index
        
        # extract samples
        X_train, y_train = X.iloc[trn_idx][label_features], y.iloc[trn_idx]
        X_valid, y_valid = X.iloc[val_idx][label_features], y.iloc[val_idx]
        X_test_label     = X_test[label_features]

        # modeling
        clf = lgb.LGBMClassifier(**CFG['lgb_params']) 
        clf = clf.fit(X_train, y_train, 
                      eval_set              = [(X_valid, y_valid)],
                      early_stopping_rounds = CFG['lgb_stop_rounds'],
                      verbose               = False)

        # prediction
        cnn_oof[val_idx, CFG['targets'].index(label)] = compute_blend(X_valid, list(X_valid.filter(like = label).columns), CFG['fold_blend'], CFG)
        lgb_oof[val_idx, CFG['targets'].index(label)] = clf.predict_proba(X_valid)[:,      1]
        lgb_tst[:, fold, CFG['targets'].index(label)] = clf.predict_proba(X_test_label)[:, 1]
        
        # performance
        cnn_auc += roc_auc_score(y_valid ,cnn_oof[val_idx, CFG['targets'].index(label)]) / CFG['lgb_folds']
        lgb_auc += roc_auc_score(y_valid, lgb_oof[val_idx, CFG['targets'].index(label)]) / CFG['lgb_folds']

    # print label performance
    print('{:<29}{:<7}{:>5.4f}'.format(label, 'CNN', cnn_auc))
    print('{:<29}{:<7}{:>5.4f}'.format(label, 'LGB', lgb_auc))
    print('-' * 45)    
    
    # replace LGB pred if CNN pred is better
    if cnn_auc >= lgb_auc:
        for fold in range(CFG['lgb_folds']):

            # extract data
            val_idx = X.loc[X['fold'] == fold].index
            X_valid, y_valid = X.iloc[val_idx][label_features], y.iloc[val_idx]

            # replace predcitions
            lgb_oof[val_idx, CFG['targets'].index(label)] = compute_blend(X_valid,      list(X_valid.filter(like      = label).columns), CFG['fold_blend'], CFG)
            lgb_tst[:, fold, CFG['targets'].index(label)] = compute_blend(X_test_label, list(X_test_label.filter(like = label).columns), CFG['fold_blend'], CFG)
            
    # store predictions
    for fold in range(CFG['lgb_folds']):
        lgb_preds     = pd.DataFrame(lgb_tst[:, fold, CFG['targets'].index(label)], columns = [str(fold) + '/' + label])
        all_lgb_preds = pd.concat([all_lgb_preds, lgb_preds], axis = 1)
    
    # add prediction as feature to next model
    if CFG['pred_as_feature']:
        X['LGB '      + label] = lgb_oof[:, CFG['targets'].index(label)]
        X_test['LGB ' + label] = np.mean(lgb_tst[:, :, CFG['targets'].index(label)], axis = 1)
        features.append('LGB ' + label)
        
# print overall performance
score_cnn = 0
score_lgb = 0
for fold in range(CFG['lgb_folds']):
    val_idx    = X.loc[X['fold'] == fold].index
    score_cnn += get_score(X.iloc[val_idx][CFG['targets']].values, cnn_oof[val_idx, :])[0] / CFG['lgb_folds']
    score_lgb += get_score(X.iloc[val_idx][CFG['targets']].values, lgb_oof[val_idx, :])[0] / CFG['lgb_folds']
print('{:<29}{:<7}{:>5.4f}'.format('OVERALL', 'CNN', score_cnn))
print('{:<29}{:<7}{:>5.4f}'.format('OVERALL', 'LGB', score_lgb))
print('-' * 45)
print('Finished in {:.2f} minutes'.format((time.time() - cv_start) / 60))

# clear memory
del cnn_oof, lgb_tst, lgb_oof, clf, X_train, X_valid, X_test, X_test_label, y_train, y_valid
del features, label_features, trn_idx, val_idx, folds, train_preds, test_preds
gc.collect()

---------------------------------------------
Label                       Model    AUC
---------------------------------------------
Swan Ganz Catheter Present   CNN    0.9990
Swan Ganz Catheter Present   LGB    0.9985
---------------------------------------------
ETT - Normal                 CNN    0.9933
ETT - Normal                 LGB    0.9933
---------------------------------------------
ETT - Abnormal               CNN    0.9876
ETT - Abnormal               LGB    0.9835
---------------------------------------------
ETT - Borderline             CNN    0.9702
ETT - Borderline             LGB    0.9709
---------------------------------------------
NGT - Abnormal               CNN    0.9718
NGT - Abnormal               LGB    0.9716
---------------------------------------------
NGT - Normal                 CNN    0.9885
NGT - Normal                 LGB    0.9882
---------------------------------------------
NGT - Incompletely Imaged    CNN    0.9861
NGT - Incompletely Imaged    LGB

147

In [22]:
####### BLEND FOLD PREDICTIONS

print('Blending fold predictions with: ' + CFG['fold_blend'])

for c in CFG['targets']:
    class_preds      = all_lgb_preds.filter(like = c).columns
    all_lgb_preds[c] = compute_blend(all_lgb_preds, class_preds, CFG['fold_blend'], CFG)
    all_lgb_preds.drop(class_preds, axis = 1, inplace = True)
           
all_lgb_preds.head()

Blending fold predictions with: pmean


,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,0.009428,0.196411,0.612663,0.001331,0.002524,0.035505,0.977584,0.041539,0.121507,0.844723,0.998810
1,0.000010,0.005762,0.000208,0.000050,0.000085,0.006471,0.000094,0.010652,0.021270,0.980424,0.000005
2,0.000044,0.005762,0.000207,0.000165,0.000205,0.006471,0.000173,0.014962,0.491136,0.492232,0.000062
3,0.001929,0.028472,0.017544,0.011476,0.009184,0.802203,0.020696,0.020465,0.070055,0.925902,0.000843
4,0.000033,0.005762,0.000602,0.000177,0.000303,0.006471,0.000580,0.010748,0.034008,0.973741,0.000014


# PUBLIC MODELS INFERENCE
In addition to 5 my CNN models, I use 2 public models:
- [SeResNet152D](https://www.kaggle.com/ammarali32/seresnet152d-cv9615) by [@ammarali32](https://www.kaggle.com/ammarali32)
- [ResNet200D](https://www.kaggle.com/ammarali32/resnet200d-public) by [@ammarali32](https://www.kaggle.com/ammarali32)

In [23]:
####### INFERENCE FOR PUBLIC MODELS 

if CFG['w_public'] > 0:

    gc.collect()

    BATCH_SIZE = 96
    IMAGE_SIZE = 640
    TEST_PATH               = '../input/ranzcr-clip-catheter-line-classification/test'
    MODEL_PATH_resnet200d   = '../input/resnet200d-public/resnet200d_320_CV9632.pth'
    MODEL_PATH_seresnet152d = '../input/seresnet152d-cv9615/seresnet152d_320_CV96.15.pth'


    class TestDataset(Dataset):
        def __init__(self, df, transform=None):
            self.df = df
            self.file_names = df['StudyInstanceUID'].values
            self.transform = transform

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            file_name = self.file_names[idx]
            file_path = f'{TEST_PATH}/{file_name}.jpg'   
            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            mask  = image > 0
            image = image[np.ix_(mask.any(1), mask.any(0))]
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.transform:
                augmented = self.transform(image=image)
                image = augmented['image']
            return image


    def get_transforms(CFG):
        return A.Compose([A.Resize(IMAGE_SIZE, IMAGE_SIZE),
                          A.Normalize(),
                          ToTensorV2()])


    class ResNet200D(nn.Module):
        def __init__(self, model_name = 'resnet200d'):
            super().__init__()
            self.model = timm.create_model(model_name, pretrained=False)
            n_features = self.model.fc.in_features
            self.model.global_pool = nn.Identity()
            self.model.fc = nn.Identity()
            self.pooling = nn.AdaptiveAvgPool2d(1)
            self.fc = nn.Linear(n_features, 11)

        def forward(self, x):
            bs = x.size(0)
            features = self.model(x)
            pooled_features = self.pooling(features).view(bs, -1)
            output = self.fc(pooled_features)
            return output


    class SeResNet152D(nn.Module):
        def __init__(self, model_name = 'seresnet152d'):
            super().__init__()
            self.model = timm.create_model(model_name, pretrained=False)
            n_features = self.model.fc.in_features
            self.model.global_pool = nn.Identity()
            self.model.fc = nn.Identity()
            self.pooling = nn.AdaptiveAvgPool2d(1)
            self.fc = nn.Linear(n_features, 11)

        def forward(self, x):
            bs = x.size(0)
            features = self.model(x)
            pooled_features = self.pooling(features).view(bs, -1)
            output = self.fc(pooled_features)
            return output


    def inference(models, test_loader, device):
        tk0 = tqdm(enumerate(test_loader), total = len(test_loader))
        probs = []
        for i, (images) in tk0:
            images = images.to(device)
            avg_preds = []
            for model in models:
                with torch.no_grad():
                    y_preds1 = model(images)
                    y_preds2 = model(images.flip(-1))
                y_preds = (y_preds1.sigmoid().to('cpu').numpy() + y_preds2.sigmoid().to('cpu').numpy()) / 2
                avg_preds.append(y_preds)
            avg_preds = np.mean(avg_preds, axis = 0)
            probs.append(avg_preds)
        probs = np.concatenate(probs)
        return probs


    models200D = []
    model = ResNet200D()
    model.load_state_dict(torch.load(MODEL_PATH_resnet200d)['model'])
    model.eval()
    model.to(device)
    models200D.append(model)
    del model

    models152D = []
    model = SeResNet152D()
    model.load_state_dict(torch.load(MODEL_PATH_seresnet152d)['model'])
    model.eval()
    model.to(device)
    models152D.append(model)
    del model
    

    test_dataset    = TestDataset(df, transform = get_transforms(CFG))
    test_loader     = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = CFG['cpu_workers'], pin_memory = True)
    predictions200d = inference(models200D, test_loader, device)
    predictions152d = inference(models152D, test_loader, device)
    
    target_cols = df.iloc[:, 1:12].columns.tolist()
    
    predictions200d = pd.DataFrame(predictions200d, columns = ['200d/' + c for c in target_cols])
    predictions152d = pd.DataFrame(predictions152d, columns = ['152d/' + c for c in target_cols])
    predictions     = pd.concat([predictions200d, predictions152d], axis = 1)
    
    df_pub = predictions.copy()
    display(df_pub.head())

100%|██████████| 1/1 [00:06<00:00,  6.89s/it]


,200d/ETT - Abnormal,200d/ETT - Borderline,200d/ETT - Normal,200d/NGT - Abnormal,200d/NGT - Borderline,200d/NGT - Incompletely Imaged,200d/NGT - Normal,200d/CVC - Abnormal,200d/CVC - Borderline,200d/CVC - Normal,200d/Swan Ganz Catheter Present,152d/ETT - Abnormal,152d/ETT - Borderline,152d/ETT - Normal,152d/NGT - Abnormal,152d/NGT - Borderline,152d/NGT - Incompletely Imaged,152d/NGT - Normal,152d/CVC - Abnormal,152d/CVC - Borderline,152d/CVC - Normal,152d/Swan Ganz Catheter Present
0,0.042606,0.621123,0.217875,0.000425,0.003781,0.011037,0.980665,0.037586,0.140550,0.889836,0.999464,0.002861,0.186532,0.791820,0.000648,0.002636,0.017136,0.980327,0.023258,0.084417,0.947521,9.992568e-01
1,0.000010,0.000032,0.000242,0.000163,0.000110,0.000041,0.000067,0.004692,0.004040,0.998147,0.000002,0.000002,0.000025,0.000052,0.000016,0.000016,0.000006,0.000010,0.021782,0.004071,0.984555,5.294575e-07
2,0.000009,0.000017,0.000151,0.000102,0.000053,0.000012,0.000028,0.003894,0.154358,0.894478,0.000004,0.000023,0.000074,0.000143,0.000262,0.000210,0.000206,0.000266,0.011191,0.344008,0.651780,2.191854e-05
3,0.001823,0.020463,0.044035,0.022296,0.011816,0.919738,0.031636,0.049662,0.051236,0.913974,0.000491,0.005439,0.067141,0.094417,0.077373,0.019847,0.876966,0.039560,0.060899,0.078361,0.737870,2.001938e-03
4,0.000026,0.000189,0.001200,0.001190,0.000670,0.000095,0.002048,0.003164,0.006033,0.991957,0.000002,0.000378,0.001077,0.000824,0.005594,0.001927,0.000676,0.002276,0.007787,0.074417,0.907453,3.312460e-06


In [24]:
####### BLEND PUBLIC MODELS

if CFG['w_public'] == 0:
    
    df_pub = all_lgb_preds.copy()
    
else:

    for c in CFG['targets']:
        class_preds = df_pub.filter(like = c).columns
        df_pub[c] = compute_blend(df_pub, class_preds, CFG['model_blend'], CFG, weights = np.array([2/3, 1/3]))
        df_pub.drop(class_preds, axis = 1, inplace = True)
        
df_pub.head()

,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,0.018603,0.421993,0.340731,0.000490,0.003357,0.012805,0.980552,0.032103,0.118895,0.908700,0.999395
1,0.000006,0.000030,0.000148,0.000079,0.000060,0.000023,0.000037,0.008018,0.004050,0.993597,0.000001
2,0.000013,0.000028,0.000148,0.000141,0.000085,0.000034,0.000062,0.005600,0.202947,0.805721,0.000007
3,0.002657,0.030849,0.057120,0.034295,0.014084,0.905275,0.034101,0.053179,0.059140,0.851432,0.000801
4,0.000068,0.000348,0.001060,0.002043,0.000964,0.000190,0.002122,0.004307,0.014879,0.963026,0.000002


# SUBMISSION

In [25]:
####### BLEND MY AND PUBLIC MODELS

all_preds = all_lgb_preds.copy()
all_preds.columns = ['my/'     + c for c in all_preds.columns]
df_pub.columns    = ['public/' + c for c in df_pub.columns]

preds = pd.concat([all_preds, df_pub], axis = 1)

for c in CFG['targets']:
    class_preds = preds.filter(like = c).columns
    preds[c] = compute_blend(preds, class_preds, CFG['model_blend'], CFG, weights = np.array([1 - CFG['w_public'], CFG['w_public']]))
    preds.drop(class_preds, axis = 1, inplace = True)

preds.head()

,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,0.011218,0.238996,0.530618,0.001045,0.002712,0.027755,0.978325,0.038970,0.120849,0.860321,0.998956
1,0.000009,0.001920,0.000191,0.000056,0.000078,0.002018,0.000075,0.009929,0.014375,0.983703,0.000004
2,0.000032,0.001904,0.000191,0.000159,0.000166,0.002158,0.000135,0.011798,0.396369,0.557929,0.000038
3,0.002092,0.029050,0.023853,0.015246,0.010236,0.826918,0.023499,0.026189,0.067167,0.906749,0.000832
4,0.000039,0.003047,0.000695,0.000343,0.000410,0.002960,0.000814,0.008612,0.027818,0.971052,0.000009


Before saving `submisssion.csv`, I check that all my model/fold combinations have succeffuly completed inference. This helps to catch memory errors that sometimes do not stop the notebook from running and can produce poor LB performance because some models have not been used. 

In [26]:
####### SUBMISSION FILE

if all_counter == len(CFG['models'] * CFG['num_folds']): 
        
    for c in CFG['targets']:
        df[c] = preds[c].rank(pct = True)
        
    df.to_csv('submission.csv', index = False)
    display(df.head())

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.96,0.96,0.78,0.72,0.82,0.84,0.96,0.68,0.44,0.62,1.00
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0.16,0.24,0.24,0.06,0.12,0.24,0.04,0.30,0.04,0.98,0.04
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0.38,0.22,0.22,0.44,0.32,0.30,0.20,0.44,0.72,0.44,0.46
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0.92,0.86,0.72,0.98,0.88,1.00,0.80,0.58,0.32,0.74,0.92
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,0.52,0.64,0.60,0.56,0.60,0.58,0.62,0.16,0.16,0.84,0.14
